In [1]:
pip install anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.3/289.3 kB 15.1 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
CLAUDE_API_KEY = userdata.get('CLAUDE_API_KEY')

In [3]:
import anthropic

client = anthropic.Client(api_key=CLAUDE_API_KEY)
MODEL_NAME = "claude-sonnet-4-20250514"

In [4]:
tools = [
    {
        "name": "get_customer_info",
        "description": "Retrieves customer information based on their customer ID. Returns the customer's name, email, and phone number.",
        "input_schema": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "string",
                    "description": "The unique identifier for the customer."
                }
            },
            "required": ["customer_id"]
        }
    },
    {
        "name": "get_order_details",
        "description": "Retrieves the details of a specific order based on the order ID. Returns the order ID, product name, quantity, price, and order status.",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The unique identifier for the order."
                }
            },
            "required": ["order_id"]
        }
    },
    {
        "name": "cancel_order",
        "description": "Cancels an order based on the provided order ID. Returns a confirmation message if the cancellation is successful.",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The unique identifier for the order to be cancelled."
                }
            },
            "required": ["order_id"]
        }
    }
]

In [5]:
def get_customer_info(customer_id):
    # Simulated customer data
    customers = {
        "C1": {"name": "John Doe", "email": "john@example.com", "phone": "123-456-7890"},
        "C2": {"name": "Jane Smith", "email": "jane@example.com", "phone": "987-654-3210"}
    }
    return customers.get(customer_id, "Customer not found")

def get_order_details(order_id):
    # Simulated order data
    orders = {
        "O1": {"id": "O1", "product": "Widget A", "quantity": 2, "price": 19.99, "status": "Shipped"},
        "O2": {"id": "O2", "product": "Gadget B", "quantity": 1, "price": 49.99, "status": "Processing"}
    }
    return orders.get(order_id, "Order not found")

def cancel_order(order_id):
    # Simulated order cancellation
    if order_id in ["O1", "O2"]:
        return True
    else:
        return False

In [6]:
def process_tool_call(tool_name, tool_input):
    if tool_name == "get_customer_info":
        return get_customer_info(tool_input["customer_id"])
    elif tool_name == "get_order_details":
        return get_order_details(tool_input["order_id"])
    elif tool_name == "cancel_order":
        return cancel_order(tool_input["order_id"])

In [7]:
import json

def chatbot_interaction(user_message):
    print(f"\n{'='*50}\nUser Message: {user_message}\n{'='*50}")

    messages = [
        {"role": "user", "content": user_message}
    ]

    response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=4096,
        tools=tools,
        messages=messages
    )

    print(f"\nInitial Response:")
    print(f"Stop Reason: {response.stop_reason}")
    print(f"Content: {response.content}")

    while response.stop_reason == "tool_use":
        tool_use = next(block for block in response.content if block.type == "tool_use")
        tool_name = tool_use.name
        tool_input = tool_use.input

        print(f"\nTool Used: {tool_name}")
        print(f"Tool Input:")
        print(json.dumps(tool_input, indent=2))

        tool_result = process_tool_call(tool_name, tool_input)

        print(f"\nTool Result:")
        print(json.dumps(tool_result, indent=2))

        messages = [
            {"role": "user", "content": user_message},
            {"role": "assistant", "content": response.content},
            {
                "role": "user",
                "content": [
                    {
                        "type": "tool_result",
                        "tool_use_id": tool_use.id,
                        "content": str(tool_result),
                    }
                ],
            },
        ]

        response = client.messages.create(
            model=MODEL_NAME,
            max_tokens=4096,
            tools=tools,
            messages=messages
        )

        print(f"\nResponse:")
        print(f"Stop Reason: {response.stop_reason}")
        print(f"Content: {response.content}")

    final_response = next(
        (block.text for block in response.content if hasattr(block, "text")),
        None,
    )

    print(f"\nFinal Response: {final_response}")

    return final_response


In [8]:
chatbot_interaction("Can you tell me the email address for customer C1?")


User Message: Can you tell me the email address for customer C1?

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(citations=None, text="I'll look up the customer information for customer C1 to get their email address.", type='text'), ToolUseBlock(id='toolu_01HaRzw4ef1i6GhA85yTx32b', input={'customer_id': 'C1'}, name='get_customer_info', type='tool_use')]

Tool Used: get_customer_info
Tool Input:
{
  "customer_id": "C1"
}

Tool Result:
{
  "name": "John Doe",
  "email": "john@example.com",
  "phone": "123-456-7890"
}

Response:
Stop Reason: end_turn
Content: [TextBlock(citations=None, text='The email address for customer C1 (John Doe) is john@example.com.', type='text')]

Final Response: The email address for customer C1 (John Doe) is john@example.com.


'The email address for customer C1 (John Doe) is john@example.com.'

In [9]:
chatbot_interaction("What is the status of order O2?")


User Message: What is the status of order O2?

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(citations=None, text="I'll check the status of order O2 for you.", type='text'), ToolUseBlock(id='toolu_01PbZ521g4R6snZ8TvsZrBse', input={'order_id': 'O2'}, name='get_order_details', type='tool_use')]

Tool Used: get_order_details
Tool Input:
{
  "order_id": "O2"
}

Tool Result:
{
  "id": "O2",
  "product": "Gadget B",
  "quantity": 1,
  "price": 49.99,
  "status": "Processing"
}

Response:
Stop Reason: end_turn
Content: [TextBlock(citations=None, text='The status of order O2 is **Processing**. Here are the complete details:\n\n- Order ID: O2\n- Product: Gadget B\n- Quantity: 1\n- Price: $49.99\n- Status: Processing\n\nYour order is currently being processed and should move to the next stage soon.', type='text')]

Final Response: The status of order O2 is **Processing**. Here are the complete details:

- Order ID: O2
- Product: Gadget B
- Quantity: 1
- Price: $49.99
- Status: Pro

'The status of order O2 is **Processing**. Here are the complete details:\n\n- Order ID: O2\n- Product: Gadget B\n- Quantity: 1\n- Price: $49.99\n- Status: Processing\n\nYour order is currently being processed and should move to the next stage soon.'

In [10]:
chatbot_interaction("Please cancel order O1 for me.")


User Message: Please cancel order O1 for me.

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(citations=None, text="I'll cancel order O1 for you right away.", type='text'), ToolUseBlock(id='toolu_016HAnoZsXBJzuSrSTzGDWAe', input={'order_id': 'O1'}, name='cancel_order', type='tool_use')]

Tool Used: cancel_order
Tool Input:
{
  "order_id": "O1"
}

Tool Result:
true

Response:
Stop Reason: end_turn
Content: [TextBlock(citations=None, text="Great! I've successfully cancelled order O1 for you. The cancellation has been processed and confirmed.", type='text')]

Final Response: Great! I've successfully cancelled order O1 for you. The cancellation has been processed and confirmed.


"Great! I've successfully cancelled order O1 for you. The cancellation has been processed and confirmed."